# Gallery Example: Γ/M/1 Queue (Gamma Arrivals)

This example demonstrates a Γ/M/1 queueing system:
- **Arrivals**: Gamma distribution inter-arrival times
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Gamma distribution provides flexible modeling of arrival patterns with adjustable variance, generalizing both Exponential and Erlang distributions.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_gamm1():    """Create Γ/M/1 queueing model"""    model = Network('Gamma/M/1')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)    sink = Sink(model, 'mySink')        # Block 2: classes    oclass = OpenClass(model, 'myClass')    # Gamma arrivals with mean=1 and shape parameter (α=2, rate=2 gives mean=1)    source.set_arrival(oclass, Gamma.fit_mean_and_order(1.0, 2))    # Exponential service with rate μ=2    queue.set_service(oclass, Exp(2))        # Block 3: topology    P = model.init_routing_matrix()    P.add_route(oclass, source, queue, 1.0)    P.add_route(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_gamm1()

## Theoretical Analysis for Γ/M/1

For Γ/M/1 with:
- **Arrival process**: Gamma with mean=1, shape=2
- **Service rate**: μ = 2 (mean service time = 0.5)
- **Utilization**: ρ = λ/μ = 1/2 = 0.5

Gamma distribution characteristics:
- **Shape parameter**: α = 2
- **Rate parameter**: β = 2 (since mean = α/β = 2/2 = 1)
- **Variance**: α/β² = 2/4 = 0.5
- **Coefficient of Variation**: C²ₐ = 1/α = 1/2 = 0.5

This is between Exponential (C²=1) and Deterministic (C²=0).

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare Gamma with Exponential and Erlang arrivals
print("\n=== Arrival Distribution Comparison ===")

def create_arrival_comparison_models():
    """Create models with different arrival distributions"""
    models = {}
    
    # Exponential arrivals (C² = 1)
    model_exp = Network('M/M/1')
    source = Source(model_exp, 'Source')
    queue = Queue(model_exp, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_exp, 'Sink')
    oclass = OpenClass(model_exp, 'Class')
    source.set_arrival(oclass, Exp(1))  # Same mean arrival rate
    queue.set_service(oclass, Exp(2))   # Same service rate
    P = model_exp.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_exp.link(P)
    models['Exponential (C²=1.0)'] = model_exp
    
    # Erlang arrivals (C² = 0.5)
    model_erl = Network('Er/M/1')
    source = Source(model_erl, 'Source')
    queue = Queue(model_erl, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_erl, 'Sink')
    oclass = OpenClass(model_erl, 'Class')
    source.set_arrival(oclass, Erlang.fit_mean_and_order(1, 2))  # C² = 1/2 = 0.5
    queue.set_service(oclass, Exp(2))
    P = model_erl.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_erl.link(P)
    models['Erlang-2 (C²=0.5)'] = model_erl
    
    # Gamma arrivals (C² = 0.5, same as Erlang-2)
    models['Gamma α=2 (C²=0.5)'] = model
    
    return models

comparison_models = create_arrival_comparison_models()

print("Arrival Distribution | C²   | Response Time | Queue Length")
print("-" * 60)

for dist_name, model_comp in comparison_models.items():
    solver_comp = MVA(model_comp)
    avg_table_comp = solver_comp.avg_table()
    
    resp_time = float(avg_table_comp.iloc[1, 2])
    queue_length = float(avg_table_comp.iloc[1, 3])
    
    # Extract CV² from distribution name
    cv_sq = dist_name.split('C²=')[1].split(')')[0] if 'C²=' in dist_name else "1.0"
    
    print(f"{dist_name:18s} | {cv_sq:4s} |     {resp_time:.4f}    |    {queue_length:.4f}")

print("\nObservations:")
print("1. Gamma and Erlang-2 give identical results (same C²)")
print("2. Lower variability (C² < 1) improves performance vs. Exponential")
print("3. Gamma distribution provides flexible modeling between Exponential and Erlang")

In [ ]:
# Analyze Gamma shape parameter effects
print("\n=== Gamma Shape Parameter Analysis ===")

def create_gamma_shape_model(alpha):
    """Create Γ/M/1 model with specified shape parameter"""
    model_shape = Network(f'Gamma-α{alpha}/M/1')
    source = Source(model_shape, 'Source')
    queue = Queue(model_shape, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_shape, 'Sink')
    
    oclass = OpenClass(model_shape, 'Class')
    source.set_arrival(oclass, Gamma.fit_mean_and_order(1.0, alpha))  # Mean=1, shape=α
    queue.set_service(oclass, Exp(2))  # Same service rate
    
    P = model_shape.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_shape.link(P)
    
    return model_shape

# Test different shape parameters
alpha_values = [0.5, 1, 2, 3, 5, 10]

print("Shape α | C² = 1/α | Response Time | Queue Length | Distribution")
print("-" * 65)

for alpha in alpha_values:
    try:
        model_alpha = create_gamma_shape_model(alpha)
        solver_alpha = MVA(model_alpha)
        avg_table_alpha = solver_alpha.avg_table()
        
        resp_time = float(avg_table_alpha.iloc[1, 2])
        queue_length = float(avg_table_alpha.iloc[1, 3])
        cv_squared = 1.0 / alpha
        
        # Special cases
        if alpha == 1:
            dist_type = "Exponential"
        elif alpha == int(alpha) and alpha > 1:
            dist_type = f"Erlang-{int(alpha)}"
        else:
            dist_type = "Gamma"
        
        print(f"  {alpha:4.1f}  |  {cv_squared:.3f}   |     {resp_time:.4f}    |    {queue_length:.4f}     | {dist_type}")
        
    except Exception as e:
        print(f"  {alpha:4.1f}  |  {1.0/alpha:.3f}   |     Error     |     Error      | {str(e)[:10]}...")

print("\nShape Parameter Effects:")
print("• α < 1: Higher variability than Exponential (more bursty)")
print("• α = 1: Equivalent to Exponential distribution")
print("• α > 1: Lower variability than Exponential (more regular)")
print("• α = integer: Equivalent to Erlang distribution")
print("• α → ∞: Approaches Deterministic distribution")

In [ ]:
# Demonstrate Gamma flexibility with different CV values
print("\n=== Gamma Flexibility: Matching Target CV² Values ===")

def create_gamma_cv_model(target_cv_squared):
    """Create Gamma model with specific CV²"""
    # For Gamma: CV² = 1/α, so α = 1/CV²
    alpha = 1.0 / target_cv_squared
    
    model_cv = Network(f'Gamma-CV²{target_cv_squared}/M/1')
    source = Source(model_cv, 'Source')
    queue = Queue(model_cv, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_cv, 'Sink')
    
    oclass = OpenClass(model_cv, 'Class')
    source.set_arrival(oclass, Gamma.fit_mean_and_order(1.0, alpha))
    queue.set_service(oclass, Exp(2))
    
    P = model_cv.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_cv.link(P)
    
    return model_cv, alpha

# Test specific CV² values
target_cv_values = [0.1, 0.25, 0.5, 1.0, 2.0, 4.0]

print("Target C² | Shape α | Response Time | Queue Length | Variability")
print("-" * 65)

for cv_sq in target_cv_values:
    try:
        model_cv, alpha = create_gamma_cv_model(cv_sq)
        solver_cv = MVA(model_cv)
        avg_table_cv = solver_cv.avg_table()
        
        resp_time = float(avg_table_cv.iloc[1, 2])
        queue_length = float(avg_table_cv.iloc[1, 3])
        
        if cv_sq < 1:
            variability = "Low (regular)"
        elif cv_sq == 1:
            variability = "Medium (exponential)"
        else:
            variability = "High (bursty)"
        
        print(f"  {cv_sq:5.2f}   |  {alpha:5.1f}  |     {resp_time:.4f}    |    {queue_length:.4f}     | {variability}")
        
    except Exception as e:
        print(f"  {cv_sq:5.2f}   |  {1.0/cv_sq:5.1f}  |     Error     |     Error      | Error")

print("\nGamma Distribution Advantages:")
print("1. Single parameter family covering wide range of variability")
print("2. Includes Exponential (α=1) and Erlang (α=integer) as special cases")
print("3. Continuous interpolation between low and high variability")
print("4. Useful for fitting real data with intermediate variability levels")